# Use the `frcmod` and `mol2` files to solvate each host-guest system with `paprika`

In [81]:
import glob as glob
import os as os
import subprocess as sp
import numpy as np

import parmed as pmd
from paprika.tleap import System

In [9]:
complexes = glob.glob('./OA-G*')

In [45]:
for complx in complexes:
    system = System()
        
    system.output_path = os.path.join(complx, 'AMBER')
    system.target_waters = 2000
    system.output_prefix = 'solvate'
    
    structure = pmd.load_file(os.path.join(complx, 'AMBER', 'complex.prmtop'))
    sodiums = len(structure[':Na+'].atoms)
    chlorides = len(structure[':Cl-'].atoms)
    
    system.neutralize = False
    system.add_ions = ['Na+', sodiums, 'Cl-', chlorides]
    system.template_lines = ['source leaprc.water.tip3p',
                           'source leaprc.gaff',
                           'loadamberparams hst.frcmod',
                           'HST = loadmol2 hst.mol2',
                           'loadamberparams gst.frcmod',
                           'GST = loadmol2 gst.mol2',
                           'model = loadpdb complex.pdb'
    ]
    
    system.build()

In [82]:
for complx in complexes:
    structure = pmd.load_file(os.path.join(complx, 'AMBER', 'solvate.prmtop'))
    
    print(f'Total charge {pmd.tools.netCharge(structure).execute():0.3f} ...')
    
    sodium = len(structure[':Na+'].atoms)
    chloride = len(structure[':Cl-'].atoms)
    
    print(f'{sodium} Na+, {chloride} Cl- in solvated structure...')
    
    hst = pmd.load_file(os.path.join(complx, 'AMBER', 'hst.mol2'), structure=True)
    gst = pmd.load_file(os.path.join(complx, 'AMBER', 'gst.mol2'), structure=True)
    
    hst_gst = hst + gst
    hst_gst_charge = pmd.tools.netCharge(hst_gst).execute()
    
    if hst_gst_charge < 0:
        sodium_added_to_neuralize = np.ceil(abs(hst_gst_charge))
        chloride_added_to_neutralize = 0
    elif hst_gst_charge > 0:
        sodium_added_to_neuralize = 0
        chloride_added_to_neutralize = np.ceil(abs(hst_gst_charge))
    else:
        sodium_added_to_neuralize = 0
        chloride_added_to_neutralize = 0
    
    print(f'{sodium_added_to_neuralize} Na+, {chloride_added_to_neutralize} Cl- for neutralization...')

    
    counterion_mass = (sodium - sodium_added_to_neuralize) * 22.9898 + \
                      (chloride - chloride_added_to_neutralize) * 35.453
    
    AMU_TO_GRAMS = 1.66054 * 10**-24
    molar_concentration = counterion_mass * (AMU_TO_GRAMS) / (22.9898 + 35.453)

    volume = structure.get_box()[0] * structure.get_box()[1] * structure.get_box()[2]
    ANGSTROM_CUBED_TO_LITERS = 1 * 10 ** -27
    liters = volume * ANGSTROM_CUBED_TO_LITERS
    
    moles_per_liter = molar_concentration / liters
    print(f'Counterion molarity {moles_per_liter:0.3f} (beyond neutralization)...')

Total charge 0.000 ...
12 Na+, 3 Cl- in solvated structure...
9.0 Na+, 0 Cl- for neutralization...
Counterion molarity 0.057 (beyond neutralization)...
Total charge 0.000 ...
12 Na+, 3 Cl- in solvated structure...
10.0 Na+, 0 Cl- for neutralization...
Counterion molarity 0.049 (beyond neutralization)...
Total charge 0.000 ...
12 Na+, 3 Cl- in solvated structure...
9.0 Na+, 0 Cl- for neutralization...
Counterion molarity 0.057 (beyond neutralization)...
Total charge 0.000 ...
12 Na+, 3 Cl- in solvated structure...
9.0 Na+, 0 Cl- for neutralization...
Counterion molarity 0.057 (beyond neutralization)...
Total charge 0.000 ...
12 Na+, 3 Cl- in solvated structure...
10.0 Na+, 0 Cl- for neutralization...
Counterion molarity 0.049 (beyond neutralization)...
Total charge 0.000 ...
12 Na+, 3 Cl- in solvated structure...
10.0 Na+, 0 Cl- for neutralization...
Counterion molarity 0.050 (beyond neutralization)...
Total charge 0.000 ...
12 Na+, 3 Cl- in solvated structure...
9.0 Na+, 0 Cl- for neut